# Assignment 2
This is an assignment for COMPSCI 690V Visual analytics. The dataset used in this assignment is from UCI machine learning repository (http://archive.ics.uci.edu/ml/datasets/Wholesale+customers). In this assignment, I will go through various methods of handling missing data in a dataset.


In [11]:
import pandas
from bokeh.layouts import row, Column
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Select
from bokeh.plotting import Figure, output_notebook, show
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource,HoverTool,CustomJS
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')



missingData = pandas.read_csv('Wholesale customers data-missing.csv')
originalData = pandas.read_csv('Wholesale customers data.csv')
missingData.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656.0,7561.0,214,2674.0,1338.0
1,2,3,7057,9810.0,9568.0,1762,3293.0,1776.0
2,2,3,6353,8808.0,7684.0,2405,3516.0,7844.0
3,1,3,13265,1196.0,4221.0,6404,507.0,1788.0
4,2,3,22615,5410.0,7198.0,3915,1777.0,5185.0


#### Check for missing values:

In [12]:
missingData.isnull().values.any()

True

In [13]:
print('Columns with missing values:')
for column in missingData.columns:
    if(missingData[column].isnull().values.any()):
        print(column)

Columns with missing values:
Milk
Grocery
Detergents_Paper
Delicassen


#### Let's visualize the dataset with the missing values:

In [14]:
xAxis = list(missingData.columns)
xAxis.remove('Channel')
xAxis.remove('Region')
yAxis = xAxis
xAxisSelect = Select(title='X-Axis', value = xAxis[0],options=xAxis)
yAxisSelect = Select(title='Y-Axis', value = yAxis[1],options=yAxis)
region1 = missingData.loc[missingData['Region'] == 1]
region2 = missingData.loc[missingData['Region'] == 2]
region3 = missingData.loc[missingData['Region'] == 3]

hover = HoverTool(tooltips=[
    (xAxisSelect.value,"$x"),
    (yAxisSelect.value,'$y')])

source1 = ColumnDataSource(data=dict(
         x=region1[xAxisSelect.value],
         y=region1[yAxisSelect.value],
     ))
source2 = ColumnDataSource(data=dict(
         x=region2[xAxisSelect.value],
         y=region2[yAxisSelect.value],
     ))
source3 = ColumnDataSource(data=dict(
         x=region3[xAxisSelect.value],
         y=region3[yAxisSelect.value],
     ))

plot = Figure(x_axis_label='X-axis',y_axis_label='Y-axis',plot_width=500, plot_height=500,
                   title='Scatter plot of the dataset with missing values', tools=[hover,'pan','box_zoom','wheel_zoom','reset'])
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][0],source = source1,legend='Lisbon')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][1],source = source2,legend='Oporto')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][2],source = source3,legend='Other Regions')
plot.legend.click_policy="hide"

xCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,region1=ColumnDataSource(region1),
                              region2=ColumnDataSource(region2),region3=ColumnDataSource(region3)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var x1 = data1['x'];
    var x2 = data2['x'];
    var x3 = data3['x'];
    

    for(var i = 0; i< x1.length; i++ ){
        x1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x2.length; i++ ){
        x2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x3.length; i++ ){
        x3[i] = region3.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
""")

yCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,region1=ColumnDataSource(region1),
                              region2=ColumnDataSource(region2),region3=ColumnDataSource(region3)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var y1 = data1['y'];
    var y2 = data2['y'];
    var y3 = data3['y'];
    

    for(var i = 0; i< y1.length; i++ ){
        y1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y2.length; i++ ){
        y2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y3.length; i++ ){
        y3[i] = region3.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
""")

xAxisSelect.js_on_change('value',xCallback)
yAxisSelect.js_on_change('value',yCallback)

output_notebook()
show(row(Column(widgetbox(xAxisSelect),widgetbox(yAxisSelect)),plot))






Loading BokehJS ...

#### The first method of handling missing data is to ignore them:


In [15]:
nullData = missingData[missingData.isnull().any(axis=1)]
nullIndexes = list(nullData.index)
fixedData = missingData.loc[~missingData.index.isin(nullIndexes)]
removedData = originalData.loc[originalData.index.isin(nullIndexes)]
xAxis = list(fixedData.columns)
xAxis.remove('Channel')
xAxis.remove('Region')
yAxis = xAxis
xAxisSelect = Select(title='X-Axis', value = xAxis[0],options=xAxis)
yAxisSelect = Select(title='Y-Axis', value = yAxis[1],options=yAxis)
region1 = fixedData.loc[fixedData['Region'] == 1]
region2 = fixedData.loc[fixedData['Region'] == 2]
region3 = fixedData.loc[fixedData['Region'] == 3]


hover = HoverTool(tooltips=[
    (xAxisSelect.value,"$x"),
    (yAxisSelect.value,'$y')])


source1 = ColumnDataSource(data=dict(
         x=region1[xAxisSelect.value],
         y=region1[yAxisSelect.value],
     ))
source2 = ColumnDataSource(data=dict(
         x=region2[xAxisSelect.value],
         y=region2[yAxisSelect.value],
     ))
source3 = ColumnDataSource(data=dict(
         x=region3[xAxisSelect.value],
         y=region3[yAxisSelect.value],
     ))
source4 = ColumnDataSource(data = dict(
        x=removedData[xAxisSelect.value],
        y=removedData[yAxisSelect.value]
    ))

plot = Figure(x_axis_label='X-axis',y_axis_label='Y-axis',plot_width=500, plot_height=500,
                   title='Scatter plot of the dataset with missing value rows removed', tools=[hover,'pan','box_zoom','wheel_zoom','reset'])
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][0],source = source1,legend='Lisbon')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][1],source = source2,legend='Oporto')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][2],source = source3,legend='Other Regions')
plot.circle('x','y',size=10,alpha=1,color = 'black',source = source4,legend='Removed Values')
plot.legend.click_policy="hide"

xCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,source4=source4,region1=ColumnDataSource(region1),
                              region2=ColumnDataSource(region2),region3=ColumnDataSource(region3),
                               removedData=ColumnDataSource(removedData)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;
    var x1 = data1['x'];
    var x2 = data2['x'];
    var x3 = data3['x'];
    var x4 = data4['x'];
    

    for(var i = 0; i< x1.length; i++ ){
        x1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x2.length; i++ ){
        x2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x3.length; i++ ){
        x3[i] = region3.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x4.length; i++ ){
        x4[i] = removedData.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();

""")

yCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,source4=source4,region1=ColumnDataSource(region1),
                              region2=ColumnDataSource(region2),region3=ColumnDataSource(region3),
                              removedData=ColumnDataSource(removedData)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;

    var y1 = data1['y'];
    var y2 = data2['y'];
    var y3 = data3['y'];
    var y4 = data4['y'];

    

    for(var i = 0; i< y1.length; i++ ){
        y1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y2.length; i++ ){
        y2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y3.length; i++ ){
        y3[i] = region3.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y4.length; i++ ){
        y4[i] = removedData.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();
""")

xAxisSelect.js_on_change('value',xCallback)
yAxisSelect.js_on_change('value',yCallback)

output_notebook()
show(row(Column(widgetbox(xAxisSelect),widgetbox(yAxisSelect)),plot))


Loading BokehJS ...

#### From the above graphs we can see that one of the points that we removed was an outlier for fresh values. 

#### The second approach to handle missing values is to assign mean values

In [16]:
print(nullData)

     Channel  Region  Fresh    Milk  Grocery  Frozen  Detergents_Paper  \
75         1       3  20398  1137.0      NaN    4407               3.0   
172        1       3    955  5479.0      NaN     333            2840.0   
180        1       3  12356     NaN   8887.0     402            1382.0   
225        1       1  12680  3243.0   4157.0     660               NaN   
274        1       3    894  1703.0   1841.0     744             759.0   
370        2       3  39679  3944.0   4955.0    1364               NaN   

     Delicassen  
75        975.0  
172       707.0  
180      2794.0  
225       786.0  
274         NaN  
370      2235.0  


In [17]:
averageData = missingData.copy()
averageData['Grocery'][75] = removedData['Grocery'].mean()
averageData['Grocery'][172] = removedData['Grocery'].mean()
averageData['Milk'][180] = removedData['Milk'].mean()
averageData['Detergents_Paper'][225] = removedData['Detergents_Paper'].mean()
averageData['Detergents_Paper'][370] = removedData['Detergents_Paper'].mean()
averageData['Delicassen'][274] = removedData['Delicassen'].mean()

In [18]:
nullIndexes = list(nullData.index)
fixedData = averageData
modifiedData = averageData.loc[averageData.index.isin(nullIndexes)]
originalRows = originalData.loc[originalData.index.isin(nullIndexes)]
xAxis = list(fixedData.columns)
xAxis.remove('Channel')
xAxis.remove('Region')
yAxis = xAxis
xAxisSelect = Select(title='X-Axis', value = xAxis[0],options=xAxis)
yAxisSelect = Select(title='Y-Axis', value = yAxis[1],options=yAxis)
region1 = fixedData.loc[fixedData['Region'] == 1]
region2 = fixedData.loc[fixedData['Region'] == 2]
region3 = fixedData.loc[fixedData['Region'] == 3]


hover = HoverTool(tooltips=[
    (xAxisSelect.value,"$x"),
    (yAxisSelect.value,'$y')])


source1 = ColumnDataSource(data=dict(
         x=region1[xAxisSelect.value],
         y=region1[yAxisSelect.value],
     ))
source2 = ColumnDataSource(data=dict(
         x=region2[xAxisSelect.value],
         y=region2[yAxisSelect.value],
     ))
source3 = ColumnDataSource(data=dict(
         x=region3[xAxisSelect.value],
         y=region3[yAxisSelect.value],
     ))
source4 = ColumnDataSource(data = dict(
        x=modifiedData[xAxisSelect.value],
        y=modifiedData[yAxisSelect.value]
    ))
source5 = ColumnDataSource(data = dict(
        x=originalRows[xAxisSelect.value],
        y=originalRows[yAxisSelect.value]
    ))

plot = Figure(x_axis_label='X-axis',y_axis_label='Y-axis',plot_width=500, plot_height=500,
                   title='Scatter plot of the dataset with missing values replaced by mean', tools=[hover,'pan','box_zoom','wheel_zoom','reset'])
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][0],source = source1,legend='Lisbon')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][1],source = source2,legend='Oporto')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][2],source = source3,legend='Other Regions')
plot.circle('x','y',size=10,alpha=1,color = 'blue',source = source5,legend='Original Values')
plot.circle('x','y',size=10,alpha=1,color = 'red',source = source4,legend='Modified Values')


plot.legend.click_policy="hide"

xCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,source4=source4,source5=source5,
                               region1=ColumnDataSource(region1),region2=ColumnDataSource(region2),
                               region3=ColumnDataSource(region3),modifiedData=ColumnDataSource(modifiedData),
                               originalRows = ColumnDataSource(originalRows)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;
    var data5 = source5.data;
    var x1 = data1['x'];
    var x2 = data2['x'];
    var x3 = data3['x'];
    var x4 = data4['x'];
    var x5 = data5['x'];

    for(var i = 0; i< x1.length; i++ ){
        x1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x2.length; i++ ){
        x2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x3.length; i++ ){
        x3[i] = region3.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x4.length; i++ ){
        x4[i] = modifiedData.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x5.length; i++ ){
        x5[i] = originalRows.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();
    source5.change.emit();

""")

yCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,source4=source4,source5=source5,
                               region1=ColumnDataSource(region1),region2=ColumnDataSource(region2),
                               region3=ColumnDataSource(region3),modifiedData=ColumnDataSource(modifiedData),
                              originalRows = ColumnDataSource(originalRows)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;
    var data5 = source5.data;


    var y1 = data1['y'];
    var y2 = data2['y'];
    var y3 = data3['y'];
    var y4 = data4['y'];
    var y5 = data5['y'];

    

    for(var i = 0; i< y1.length; i++ ){
        y1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y2.length; i++ ){
        y2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y3.length; i++ ){
        y3[i] = region3.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y4.length; i++ ){
        y4[i] = modifiedData.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y5.length; i++ ){
        y5[i] = originalRows.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();
    source5.change.emit();
""")

xAxisSelect.js_on_change('value',xCallback)
yAxisSelect.js_on_change('value',yCallback)

output_notebook()
show(row(Column(widgetbox(xAxisSelect),widgetbox(yAxisSelect)),plot))

Loading BokehJS ...

#### From the above graphs we can see that the mean value is very near to the original values in most cases.

#### next we will try to replace values by linear regressions. 
From the graph above:
1. Grocery forms almost a linear relationship with detergents_paper
2. Milk forms almost a linear relationship with detergents_paper
3. Detergents_paper forms almost a linear relationship with Grocery
4. Dessican paper does not seem for have any relation with other columns so we will use mean


In [19]:
Grocery = removedData["Grocery"]
Detergents_Paper = removedData["Detergents_Paper"]
Milk = removedData["Milk"]

model1 = sm.OLS(Detergents_Paper, Grocery).fit()
predictedData = missingData.copy()
predictedData['Grocery'][75] = model1.predict(predictedData['Detergents_Paper'][75])
predictedData['Grocery'][172] = model1.predict(predictedData['Detergents_Paper'][172])

model2 = sm.OLS(Detergents_Paper, Milk).fit()
predictedData['Milk'][180] = model2.predict(predictedData['Detergents_Paper'][180])

model3 = sm.OLS(Grocery, Detergents_Paper).fit()
predictedData['Detergents_Paper'][225] = model3.predict(predictedData['Grocery'][225])
predictedData['Detergents_Paper'][370] = model3.predict(predictedData['Grocery'][370])
predictedData['Delicassen'][274] = removedData['Delicassen'].mean()


In [20]:
nullIndexes = list(nullData.index)
fixedData = predictedData
modifiedData = predictedData.loc[predictedData.index.isin(nullIndexes)]
originalRows = originalData.loc[originalData.index.isin(nullIndexes)]
xAxis = list(fixedData.columns)
xAxis.remove('Channel')
xAxis.remove('Region')
yAxis = xAxis
xAxisSelect = Select(title='X-Axis', value = xAxis[0],options=xAxis)
yAxisSelect = Select(title='Y-Axis', value = yAxis[1],options=yAxis)
region1 = fixedData.loc[fixedData['Region'] == 1]
region2 = fixedData.loc[fixedData['Region'] == 2]
region3 = fixedData.loc[fixedData['Region'] == 3]


hover = HoverTool(tooltips=[
    (xAxisSelect.value,"$x"),
    (yAxisSelect.value,'$y')])


source1 = ColumnDataSource(data=dict(
         x=region1[xAxisSelect.value],
         y=region1[yAxisSelect.value],
     ))
source2 = ColumnDataSource(data=dict(
         x=region2[xAxisSelect.value],
         y=region2[yAxisSelect.value],
     ))
source3 = ColumnDataSource(data=dict(
         x=region3[xAxisSelect.value],
         y=region3[yAxisSelect.value],
     ))
source4 = ColumnDataSource(data = dict(
        x=modifiedData[xAxisSelect.value],
        y=modifiedData[yAxisSelect.value]
    ))
source5 = ColumnDataSource(data = dict(
        x=originalRows[xAxisSelect.value],
        y=originalRows[yAxisSelect.value]
    ))

plot = Figure(x_axis_label='X-axis',y_axis_label='Y-axis',plot_width=500, plot_height=500,
                   title='Scatter plot of the dataset with missing values replaced by mean', tools=[hover,'pan','box_zoom','wheel_zoom','reset'])
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][0],source = source1,legend='Lisbon')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][1],source = source2,legend='Oporto')
plot.circle('x','y',size=10,alpha=.2,color = Category10[3][2],source = source3,legend='Other Regions')
plot.circle('x','y',size=10,alpha=1,color = 'blue',source = source5,legend='Original Values')
plot.circle('x','y',size=10,alpha=1,color = 'red',source = source4,legend='Modified Values')


plot.legend.click_policy="hide"

xCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,source4=source4,source5=source5,
                               region1=ColumnDataSource(region1),region2=ColumnDataSource(region2),
                               region3=ColumnDataSource(region3),modifiedData=ColumnDataSource(modifiedData),
                               originalRows = ColumnDataSource(originalRows)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;
    var data5 = source5.data;
    var x1 = data1['x'];
    var x2 = data2['x'];
    var x3 = data3['x'];
    var x4 = data4['x'];
    var x5 = data5['x'];

    for(var i = 0; i< x1.length; i++ ){
        x1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x2.length; i++ ){
        x2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x3.length; i++ ){
        x3[i] = region3.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x4.length; i++ ){
        x4[i] = modifiedData.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< x5.length; i++ ){
        x5[i] = originalRows.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();
    source5.change.emit();

""")

yCallback = CustomJS(args=dict(source1=source1,source2=source2,source3=source3,source4=source4,source5=source5,
                               region1=ColumnDataSource(region1),region2=ColumnDataSource(region2),
                               region3=ColumnDataSource(region3),modifiedData=ColumnDataSource(modifiedData),
                              originalRows = ColumnDataSource(originalRows)), code="""
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;
    var data5 = source5.data;


    var y1 = data1['y'];
    var y2 = data2['y'];
    var y3 = data3['y'];
    var y4 = data4['y'];
    var y5 = data5['y'];

    

    for(var i = 0; i< y1.length; i++ ){
        y1[i] = region1.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y2.length; i++ ){
        y2[i] = region2.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y3.length; i++ ){
        y3[i] = region3.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y4.length; i++ ){
        y4[i] = modifiedData.data[cb_obj.value.toString()][i];
    }
    for(var i = 0; i< y5.length; i++ ){
        y5[i] = originalRows.data[cb_obj.value.toString()][i];
    }
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();
    source5.change.emit();
""")

xAxisSelect.js_on_change('value',xCallback)
yAxisSelect.js_on_change('value',yCallback)

output_notebook()
show(row(Column(widgetbox(xAxisSelect),widgetbox(yAxisSelect)),plot))

Loading BokehJS ...

#### As we can see from the graphs above, some of the values are very far from the original values. Therefore linear regression is not viable for these values.